### Web Scraping

In [6]:
## importing pre-requisites for web scraping

from bs4 import BeautifulSoup
import requests

# loading the link for data scraping 
url = 'https://www.sec.gov/foia/iareports/inva-archive.htm'
r  = requests.get(url)
data = r.text
soup = BeautifulSoup(data)
ls= []
for link in soup.find_all('a'):
    ls.append((link.get('href'),link.string))

In [7]:
## scraped data

ls[:10]

[('/index.htm', None),
 ('/edgar/searchedgar/companysearch.html', u'Company Filings'),
 ('/search/search.htm', u'More Search Options'),
 ('#main-content', u'Skip to Main Content'),
 ('/about.shtml', u'About'),
 ('/about/whatwedo.shtml', u'What We Do'),
 ('/about/commissioner.shtml', u'Commissioners'),
 ('/about/laws.shtml', u'Securities Laws'),
 ('/reports', u'Reports and Publications'),
 ('/about/sec-docket.shtml', u'SEC Docket')]

In [8]:
## understanding the pattern and picking the required information
c=0
for i in ls:
    if 'zip' in i[0]:print c;break
    c+=1
c=0
for i in ls:
    if i==('/foia/iareports/ia060506.zip', u'June 2006'):print c;break
    c+=1 
c=0
for i in ls:
    if i==('/foia/iareports/ia020112-exempt.zip', u'Exempt Reporting Advisers'):print c;break
    c+=1

62
244
191


### Preparing Datasets

In [9]:
dates,file_name,type_ =[],[],[]
for i in xrange(62,192,2):
    dates.append(ls[i][1])
    file_name.append(ls[i][0])
    type_.append('exempt')
for i in xrange(192,245,1):
    dates.append(ls[i][1])
    file_name.append(ls[i][0])
    type_.append('Non-exempt')

In [5]:
## (a)function to create the Dataframe

import pandas as pd, numpy as np

def sec_dataframe(files,date,types):
    columns = ['File_URL','Date','type']
    this_is_df = pd.DataFrame(columns=columns)
    this_is_df['File_URL'] = files
    this_is_df['Date'] = date
    this_is_df['type'] = types
    mapping ={'January':'01','February':'02','March':'03','April':'04','May':'05','June':'06','July':'07','August':'08','September':'09','October':'10','November':'11','December':'12',}
    new_dates = []
    for i in this_is_df.Date:
        month = mapping[i.split()[0]]
        year = i[::-1][:4][::-1]
        new_dates.append(year+'-'+month)
    this_is_df['Date'] = new_dates
    return this_is_df


In [10]:
## running the sec_dataframe function to create dataset

result = sec_dataframe(file_name,dates,type_)


In [11]:
##output

result.head()

,File_URL,Date,type
0,/foia/iareports/ia050117.zip,2017-05,exempt
1,/foia/iareports/ia040317.zip,2017-04,exempt
2,/foia/iareports/ia030117.zip,2017-03,exempt
3,/foia/iareports/ia020117.zip,2017-02,exempt
4,/foia/iareports/ia020117.zip,2017-02,exempt


In [18]:
## (b)function to perform a query for parsing the zip file - get_sec_zip_by_period
## test run of this function is done to generate just one output for demonstration purpose

import urllib2
import zipfile
import os
from datetime import datetime
from pprint import pprint


## storing current working directory for future purpose
current = os.getcwd()


def get_sec_zip_by_period(df, period, is_exempt = False, only_most_recent=False):
    
    urls = [] # for storing urls after specific query
    
    if only_most_recent==False:
        for i in period:
            target_samples = df.loc[df['Date'] == i]
            for j in target_samples.File_URL:
                urls.append(j)
    else:
        print "will be using most recent period"
        
        ## to calculate the most recent period 
        times = df.Date
        times_= [datetime.strptime(time, "%Y-%m") for time in times]
        times_.sort(reverse=True)
        recent_period = [time.strftime('%Y-%m') for time in times_][0]
        
        ## according to is_exempt status
        if is_exempt==False: type_ = 'Non-exempt'
        else: type_='exempt'
        target_samples = df.loc[(df['Date']==recent_period) & (df['type']==type_)]
        for url in target_samples.File_URL:
            urls.append(url)
    
    
    for link in urls:
        
        complete = 'https://www.sec.gov'+link                           # download the zip file from the query link
        response = urllib2.urlopen(complete)
        zipcontent= response.read()
        
        zip_name = complete[::-1][:complete[::-1].index('/')][::-1]     # name of the zip file
        print "opening link..."
            
        with open(zip_name, 'w') as f:                                  # opening the zip file  
            f.write(zipcontent)
        print "extracting..."
        zip_ref = zipfile.ZipFile(zip_name, 'r')
                                                    
        folder_name_to_save_to = 'new'                                  # will be saved to this folder
        
        zip_ref.extractall(folder_name_to_save_to)                      # extraction of files from the zip file here
        zip_ref.close()
        
        print "opening file..."
        
        mypath = current+'/'+folder_name_to_save_to                     # complete path to access the file
        
        for f in os.listdir(mypath):
            print f 
            if 'xlsx' in f.lower():                                     # just to check which file it is(xlsx/csv) 
                parsed_file = pd.read_excel(mypath+'/'+str(f)) 
            elif 'csv' in f.lower():
                parsed_file = pd.read_csv(mypath+'/'+str(f))            # reading the xlsx/csv file (parsing)
            
            pprint (parsed_file.head())                                 # print the xlsx/csv file 
            break    # uncommented just for demonstration
        break        # uncommented just for demonstration

In [19]:
### Test Run of the second function (b) based on the parameters as shown

## we can see IA FOIA Download 2-1-10.CSV is parsed (the website contains xlsx and csvs as well)


get_sec_zip_by_period(result,period=["2010-02"],is_exempt=True,only_most_recent=True)

will be using most recent period
opening link...
extracting...
opening file...
IA FOIA Download 2-1-10.CSV
  SEC Region  Organization CRD#       SEC#              Primary Business Name  \
0       CHRO               7365  801-39365              WRP INVESTMENTS, INC.   
1        ARO             108741  801-34451  SOUTHERNSUN ASSET MANAGEMENT, INC   
2       SFRO             114093  801-62709      LIGHTHOUSE FINANCIAL PLANNING   
3        BRO             150778  801-70371             TRADEX CAPITAL MARKETS   
4       LARO             150220  801-70152                  ARMORED WOLF, LLC   

                           Legal Name     Main Office Street Address 1  \
0               WRP INVESTMENTS, INC.                 4407 BELMONT AVE   
1   SOUTHERNSUN ASSET MANAGEMENT, INC               6000 POPLAR AVENUE   
2  LIGHTHOUSE FINANCIAL PLANNING, LLC  101 PARK SHORE DRIVE, SUITE 100   
3        TRADEX CAPITAL MARKETS, LLC.                  35 MASON STREET   
4                   ARMORED WOLF, LL

In [20]:
## explicitly loading the csv downloaded by the above function for BlackStone entities (for further investigation)

df = pd.read_csv('/home/abhishek/Desktop/new/IA FOIA Download 2-1-10.CSV')

In [21]:
## Analysis of Blackstone Entities


'''
based on the facts we know about blackstone we can narrow our search as shown below
potential blackstone companies features : website-> www.blackstone.com,
primary office-> 345 Park Avenue, company name -> BLACKSTONE REAL ESTATE 

observation: possible 26 blackstone companines

complete string matching for legal name not possible so matching substring
'''


analysis = df.loc[(df['World Wide Web Site Address']=='WWW.BLACKSTONE.COM') | \
       (df['Main Office Street Address 1']=='345 PARK AVENUE') | \
       (df['Legal Name'].str.contains("BLACKSTONE REAL ESTATE"))\
      ]
analysis

,SEC Region,Organization CRD#,SEC#,Primary Business Name,Legal Name,Main Office Street Address 1,Main Office Street Address 2,Main Office City,Main Office State,Main Office Country,...,11E(1),11E(2),11E(3),11E(4),11F,11G,11H(1)(a),11H(1)(b),11H(1)(c),11H(2)
630,NYRO,120934,801-61467,BLACKSTONE DEBT ADVISORS L.P.,BLACKSTONE DEBT ADVISORS L.P.,280 PARK AVENUE,NaN,NEW YORK,NY,UNITED STATES,...,N,N,N,N,N,N,N,N,N,N
2389,NYRO,136991,801-64715,BLACKSTONE ASIA ADVISORS L.L.C.,BLACKSTONE ASIA ADVISORS L.L.C.,345 PARK AVENUE,NaN,NEW YORK,NY,USA,...,N,N,N,N,N,N,N,N,N,N
2390,NYRO,109106,801-57640,BLACKSTONE COMMUNICATIONS ADVISORS I LLC,BLACKSTONE COMMUNICATIONS ADVISORS I LLC,345 PARK AVENUE,NaN,NEW YORK,NY,USA,...,N,N,N,N,N,N,N,N,N,N
2391,NYRO,109108,801-55275,BLACKSTONE MANAGEMENT PARTNERS LP,BLACKSTONE MANAGEMENT PARTNERS LP,345 PARK AVENUE,NaN,NEW YORK,NY,USA,...,N,N,N,N,N,N,N,N,N,N
2392,NYRO,109103,801-54696,BLACKSTONE MANAGEMENT PARTNERS III LLC,BLACKSTONE MANAGEMENT PARTNERS III LLC,345 PARK AVENUE,NaN,NEW YORK,NY,USA,...,N,N,N,N,N,N,N,N,N,N
2393,NYRO,114716,801-60568,BLACKSTONE MANAGEMENT PARTNERS IV L.L.C.,BLACKSTONE MANAGEMENT PARTNERS IV L.L.C.,345 PARK AVENUE,NaN,NEW YORK,NY,USA,...,N,N,N,N,N,N,N,N,N,N
2394,NYRO,136979,801-64755,BLACKSTONE MANAGEMENT PARTNERS L.L.C.,BLACKSTONE MANAGEMENT PARTNERS L.L.C.,345 PARK AVENUE,NaN,NEW YORK,NY,USA,...,N,N,N,N,N,N,N,N,N,N
2395,NYRO,138208,801-64992,BLACKSTONE REAL ESTATE ADVISORS V L.P.,BLACKSTONE REAL ESTATE ADVISORS V L.P.,345 PARK AVENUE,NaN,NEW YORK,NY,USA,...,N,N,N,N,N,N,N,N,N,N
2396,NYRO,142979,801-67509,BLACKSTONE REAL ESTATE ADVISORS VI L.P.,BLACKSTONE REAL ESTATE ADVISORS VI L.P.,345 PARK AVENUE,NaN,NEW YORK,NY,USA,...,N,N,N,N,N,N,N,N,N,N
3074,NYRO,107580,801-52559,BLACKSTONE ALTERNATIVE ASSET MANAGEMENT LP,BLACKSTONE ALTERNATIVE ASSET MANAGEMENT LP,345 PARK AVE,NaN,NEW YORK,NY,UNITED STATES,...,N,N,N,N,N,N,N,N,N,N


In [22]:
## the master file that contains the name of blackstone entities

## reading the json from the link provided

d2 = pd.read_json('https://doppler.finra.org/doppler-lookup/api/v1/search/firms?hl=true&nrows=99000&query=%20Blackstone&r=2500&wt=json')

In [23]:
## just for analysing the structure of the json file

d2['results']['BROKER_CHECK_FIRM']['results'][23]['fields']

{u'bc_branches_count': 0,
 u'bc_firm_name': u'BLACKSTONE DISTRESSED SECURITIES ADVISORS L.P.',
 u'bc_ia_address_details': u'{"officeAddress":{"street1":"345 PARK AVENUE","city":"NEW YORK","state":"NY","country":"UNITED STATES","postalCode":"10154","street2":"31ST FLOOR"}}',
 u'bc_ia_scope': u'INACTIVE',
 u'bc_other_names': [u'BLACKSTONE DISTRESSED DEBT ADVISORS L.P.',
  u'BLACKSTONE DISTRESSED SECURITIES ADVISORS L.P.'],
 u'bc_sec_number': u'64038',
 u'bc_source_id': u'134013',
 u'score': 0.375}

In [29]:
## creating a dataframe for the firm_name, src_id and score

blackstone = pd.DataFrame(columns = ['bc_firm_name','bc_source_id','score'])

In [30]:
## on checking the dimensions of the json file preparing the master file dataframe

firm,ids,score=[],[],[]

for i in xrange(24):
    firm.append(d2['results']['BROKER_CHECK_FIRM']['results'][i]['fields']['bc_firm_name'])
    ids.append(int(d2['results']['BROKER_CHECK_FIRM']['results'][i]['fields']['bc_source_id']))
    score.append(d2['results']['BROKER_CHECK_FIRM']['results'][i]['fields']['score'])

In [31]:
## populating the dataframe 

blackstone.bc_firm_name = firm
blackstone.bc_source_id = ids
blackstone.score = score

In [32]:
## getting the dataframe columns with score values of equal and above of 0.40

blackstone.loc[blacstone['score']>=0.4]

,bc_firm_name,bc_source_id,score
0,BLACKSTONE MEZZANINE ADVISORS LP,109099,0.5000
1,BLACKSTONE ENERGY CORPORATION,14098,0.5000
2,BLACKSTONE WEALTH MANAGEMENT,160794,0.5000
3,BLACKSTONE MANAGEMENT PARTNERS LP,109108,0.5000
4,BLACKSTONE REAL ESTATE ADVISORS LP,109110,0.4375
5,BLACKSTONE COMMUNICATIONS ADVISORS I LLC,109106,0.4375
6,BLACKSTONE MANAGEMENT PARTNERS III LLC,109103,0.4375
7,BLACKSTONE ADVISORY PARTNERS L.P.,17917,0.4375
8,BLACKSTONE PROPERTY ADVISORS L.P.,226598,0.4375
9,BLACKSTONE ALTERNATIVE INVESTMENT ADVISORS LLC,165782,0.4375


In [35]:
## now chcking the amount of blackstone entites we were able to find with this dataset
## the dataset we compare is a different to this json file so 100% match wont be possible as some companies might miss out

## creating a set to check for common source ids in master_file dataset and our analysis dataset 

## observation : the facts hold and we get a total of 11 blackstone entities in our prepared dataset(on comparision with mastr file)

ids1 = set(list(analysis['Organization CRD#']))
ids2 = set(list(blackstone['bc_source_id']))


commmon_entities = ids1.intersection(ids2)

print len(commmon_entities)
    

11


In [40]:
## checking which entities where those
for i in list(commmon_entities):
    print blackstone.loc[blackstone['bc_source_id']==i]['bc_firm_name']

22    BLACKSTONE REAL ESTATE ADVISORS V L.P.
Name: bc_firm_name, dtype: object
11    BLACKSTONE DEBT ADVISORS L.P.
Name: bc_firm_name, dtype: object
10    BLACKSTONE ALTERNATIVE ASSET MANAGEMENT LP
Name: bc_firm_name, dtype: object
0    BLACKSTONE MEZZANINE ADVISORS LP
Name: bc_firm_name, dtype: object
12    BLACKSTONE REAL ESTATE ADVISORS III LP
Name: bc_firm_name, dtype: object
6    BLACKSTONE MANAGEMENT PARTNERS III LLC
Name: bc_firm_name, dtype: object
21    BLACKSTONE REAL ESTATE ADVISORS EUROPE L.P.
Name: bc_firm_name, dtype: object
5    BLACKSTONE COMMUNICATIONS ADVISORS I LLC
Name: bc_firm_name, dtype: object
3    BLACKSTONE MANAGEMENT PARTNERS LP
Name: bc_firm_name, dtype: object
20    BLACKSTONE MANAGEMENT PARTNERS IV L.L.C.
Name: bc_firm_name, dtype: object
23    BLACKSTONE DISTRESSED SECURITIES ADVISORS L.P.
Name: bc_firm_name, dtype: object
